In [5]:
from rrt_star import *
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import roboticstoolbox as rtb
import spatialmath as sm # provides objects for representing transformations
from swift import Swift # lightweight browser-based simulator which comes with the toolbox
import spatialgeometry as sg # utility package for dealing with geometric objects

In [8]:
robot = rtb.models.wx200()
print(robot)
useful_links = lambda robot: [robot.links[i] for i in (0,1,2,3,4)]

ERobot: wx200 (by Interbotix), 8 joints (RRRRRRPP), 4 branches, dynamics, geometry, collision
┌──────┬─────────────────────┬───────┬───────────────────┬───────────────────────────────────────────┐
│ link │        link         │ joint │      parent       │            ETS: parent to link            │
├──────┼─────────────────────┼───────┼───────────────────┼───────────────────────────────────────────┤
│    0 │ /base_link          │       │ BASE              │ SE3()                                     │
│    1 │ /shoulder_link      │     0 │ /base_link        │ SE3(0, 0, 0.0716) ⊕ Rz(q0)                │
│    2 │ /upper_arm_link     │     1 │ /shoulder_link    │ SE3(0, 0, 0.03865) ⊕ Ry(q1)               │
│    3 │ /forearm_link       │     2 │ /upper_arm_link   │ SE3(0.05, 0, 0.2; 180°, -0°, 0°) ⊕ Ry(q2) │
│    4 │ /wrist_link         │     3 │ /forearm_link     │ SE3(0.2, 0, 0) ⊕ Ry(q3)                   │
│    5 │ /gripper_link       │     4 │ /wrist_link       │ SE3(0.065, 0, 0; -180°,

In [9]:
def has_pairwise_collision(links):
	n_links = len(links)
	for i1 in range(n_links):
		link1 = links[i1]
		for i2 in list(range(0,i1-1))+list(range(i1+2,n_links)):
			link2 = links[i2]
			for collider1 in link1.collision:
				if collider1.stype == 'mesh':
					continue
				for collider2 in link2.collision:
					if collider2.stype == 'mesh':
						continue
					if collider1.iscollided(collider2):
						print(collider1, '\n with \n', collider2)
						return True

	return False

has_pairwise_collision(useful_links(robot))

False

In [12]:
start_config = [0, 0, 0, 0, 0, 0]  # Initial 6D configuration
goal_config = [4, 4, np.pi/4, np.pi/3, -np.pi/6, np.pi/2]  # Goal 6D configuration

rrt_star = RRTStar(start=start_config, goal=goal_config)
path = rrt_star.rrt_star()

if path:
	print("Found path:")
	for config in path:
		print(config)
else:
	print("Path not found.")

Goal reached in 155 iterations.
Found path:
[0, 0, 0, 0, 0, 0]
[0.21181259516888687, -0.27174345627430885, -0.21937765680249494, -0.12348404309787679, 0.24025527986786538, 0.10096263562118481]
[0.9176338778960929, 0.02495947700553075, -0.6706102633928002, -0.25285098959727753, 0.5812092078723523, 0.11992856225760434]
[1.5966497851907748, 0.40612743742021545, -0.6373491556729917, -0.6056803100440354, 0.151279682849491, 0.015806014552707456]
[2.0326489759280704, 0.99320685796408, -0.37019416713807674, -0.5790274193871967, 0.33010594244958935, -0.07510677463308332]
[2.1786464154971856, 1.7290132846263715, -0.2306857717272085, -0.6914154272280709, 0.6277800972035504, 0.05234898409058117]
[2.4051768555836004, 2.0114667436588314, -0.10431056213200154, -0.4751758313068848, 0.48457761033815616, 0.24120552887002467]
[2.8598371991206917, 2.2595332728623956, -0.19082404674151815, -0.2588959324095693, 0.45053252442037567, 0.7026591046200269]
[3.053216506720144, 2.5547281786549605, -0.0252501699983

In [13]:
robot.q

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
ee_axes = sg.Axes(0.1)
goal_axes = sg.Axes(0.1)

# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to the ready position
robot._T = np.diag(np.ones(4))
robot.qr = np.zeros(8)
robot.q = robot.qr

# Step the sim to view the robot in this configuration
env.step(0)

action_size = 4

# Specify our timestep
dt = 0.5

# # # turn from rightward to forward
# robot._T = robot._T * sm.SE3.RPY(0.0, 0.0, -np.pi/2)
# Tep = sm.SE3.RPY(0.0, 0.0, np.pi/2) * sm.SE3.Trans(0.2, 0.0, 0.2)
# robot.q[0] = np.pi/3

# Set the goal axes to Tep
# goal_axes.T = Tep

time = 0
# error_history = []
max_qd = np.zeros(8)

# Run the simulation for duration seconds
for step_i in range(len(path)):
	robot.q = np.hstack([np.zeros(2), path[step_i]])

	# Update the ee axes
	ee_axes.T = robot.fkine(robot.q, end='/ee_arm_link').A
	
	# Step the simulator by dt seconds
	env.step(dt)
	time += dt